In [2]:
import cobra
from cobra.io import load_model
from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import logging
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.io import write_sbml_model, validate_sbml_model

#### READ PVUL MODEL AND GET BLOCKED RXNS

In [3]:
%cd C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\OTHER_SPECIES\Phaseolus vulgaris
pvulModel=read_sbml_model('pvulModel_exchange&transportrxn.xml')
resPvulModel=cobra.flux_analysis.find_blocked_reactions(pvulModel)
len(resPvulModel)

C:\Users\duart\OneDrive\Documentos\UNIANDES\PHASEOLUS LUNATUS\MODELS\OTHER_SPECIES\Phaseolus vulgaris


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


2037

#### GET UNBLOCKED RXNS

In [4]:
reactionsTotal=set({})
for i in pvulModel.reactions:
    reactionsTotal.add(i.id)
unblocked_reactions = reactionsTotal - set(resPvulModel)


In [5]:
rxnsubmodel=[]
for i in unblocked_reactions:
    if pvulModel.reactions.__contains__(i):
        rxnsubmodel.append(pvulModel.reactions.get_by_id(i))
        


#### CREATE A SUBMODEL OF UNBLOCKED RXNS

In [6]:
submodel = Model("submodel")
submodel.add_reactions(rxnsubmodel)
submodel


Name,submodel
Memory address,24f84d05db0
Number of metabolites,688
Number of reactions,1050
Number of genes,2144
Number of groups,0
Objective expression,0
Compartments,"c, e"


In [7]:
blocked=cobra.flux_analysis.find_blocked_reactions(submodel)

In [8]:
import pandas as pd
submodel.objective="RXN-17121"
sol=submodel.optimize()
sol_df=pd.DataFrame(sol.fluxes)
print(sol)



<Solution 1000.000 at 0x24f9dfeb940>


In [9]:
write_sbml_model(submodel, "submodelUnblocked.xml")